# Math 753/853 HW2: Mathematics in finite precision

## Before you begin!

**First rename this file "math753-hw2-mylastname"** using your last name in the indicated spot. With the file extension, the full filename should be "math753-hw2-mylastname.ipynb". No capital letters, please. 

**To submit the homework** email it to `john.gibson@unh.edu` with the subject "MATH 753 HW2" by midnight, Sunday Oct 1, 2017.

**Writing explanations in your solutions.**  Many of the problems ask for *explanations* of answers and calculations. For these you should write one to several complete sentences that explain the reasoning behind your work in a manner that would be helpful to a fellow student.

## Problem 0: example with solution

Use the Conditioning and Accuracy Theorem to estimate a value for the expected relative error $|\tilde{f} - f|\,/\,|f|$ for the floating-point calculation $\tilde{f}(x)$ of the mathematical problem $f(x)$ in 64-bit arithmetic. Then devise a calculation in Julia that confirms your expectations. Explain each of your answers.

**Subtraction, very different numbers.** Estimate a value for the relative error for floating-point calculation of subtraction, $f(x_1, x_2) = x_2 - x_1$, using $x_1 = 17/13 \times 10^{23}$ and $x_2 = 19/57$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.

In [2]:
# The condition number of positive floating-point subtraction f(x₁, x₂) = x₂ - x₁ is
# κ = max(x₁, x₂)/|x₂ - x₁| 
# 
# for x₁ = 17/13 × 10²³ and x₂ = 19/57, this works out to 
#
# κ(x) = 17/13 × 10²³ / |17/13 × 10²³ - 19/57| ≈ 1
#
# The conditioning and accuracy theorem then estimates the relative error 
# of the 64-bit calculation to be

# |f̃ - f|/|f| = O(κ(x) ϵₘ) ≈ 2e-16

# Let's verify that with a direct calculation in Julia. Use BigFloats to represent the
# "real" numbers in high precision and compute the "true" value f, and then calculate
# the floating-point value f̃ in Float64. 

x₁ = BigFloat(17//13) * BigFloat(10)^23
x₂ = BigFloat(19//57)

f = x₂ - x₁

f̃ = Float64(x₂) - Float64(x₁)

abs(f̃ - f)/abs(f)

3.454132960784313725490204883084017685505574778953623547496062039450477003140082e-17

The relative error is about 3e-17, which slightly smaller than the estimate with machine precision 2e-16. 

## Problem 1

What is the next `Float64` number after 6.0? Determine the precise value based on the structure of the 64-bit floating point number system as presented in lecture. Explain your reasoning. Then confirm your answer with a few calculations in Julia, and explain how the calculations confirm your expectations. 

In [5]:
# In Float64, the next number after 1 is 1 + 2^-52, since there are 52 bits allocated to the mantissa. 
# So for numbers between 1 and 2, the spacing is 2^-52. Larger numbers are built by multiplying these by powers of 2.
# So for numbers between 2 and 4, the spacing is 2 × 2^-52 == 2^-51
# So for numbers between 4 and 8, the spacing is 4 × 2^-52 == 2^-50. Let's try that out

6.0 + 2.0^-50

6.000000000000001

In [6]:
(6.0 + 2.0^-50) - 6.0

8.881784197001252e-16

In [7]:
2.0^-50

8.881784197001252e-16

In [8]:
# Yep, we can definitely represent 6 + 2^-50. Can we fit another number between 6 and that? Let's try
6.0 + 2.0^-51

6.0

In [9]:
# Doesn't look like it. Try subtracting off 6 to see exactly what the difference is
(6.0 + 2.0^-51) - 6.0

0.0

In [10]:
# Ok, that shows that 6.0 + 2.0^-51 rounds down to 6.0 exactly.
# Julia actually has a convenient function for finding the next float after a given number
nextfloat(6.0)

6.000000000000001

In [11]:
# and we can see the difference by subtracting 
nextfloat(6.0) - 6.0

8.881784197001252e-16

In [12]:
# Again, that is 2^-50. So the next Float64 after 6 is 6 + 2^-50

## Problem 2

What range of integers can be represented exactly in the `Float64` number system? I.e. what is the maximum value of $N$ for which all integers between $-N$ and $N$ are represented exactly as `Float64`s? As before, determine the answer based on the structure of the 64-bit floating-point number system as presented in lecture, and explain your reasoning. Then confirm your answer with a few calculations in Julia, and explain how the calculations confirm your expectations. 

In [13]:
# In lecture we saw that Float 64s had form
#
# sign * (1 + b1 2^-1 + b2 2^-2 + ... + b52 2^-52) * 2^e
# 
# Transfer 2^n from 2^e onto the sum to get
#
# sign * (2^n + b1 2^(n-1) + b2 2^(n-2) + ... + b52 2^(n-52)) 2^(e-n)
#
# So you can produce any integer up to 2^53 - 1 by figuring out its binary expansion
# and encoding it in the above form, where 2^n is the leading value in the binary expansion
# and e=n so that 2^(e-n) = 1
#
#     binary     
# 1 =   1    =   2^0
# 2 =  10    =   2^1
# 3 =  11    =   2^1 + 2^0
# 4 = 100    =   2^2 +  0 
# 5 = 101    =   2^2 +  0  + 2^0
# 6 = 110    =   2^2 + 2^1 +  0 
# 7 = 111    =   2^2 + 2^1 + 2^0
#
# etc. That works up to n=52
#
# 2^53 - 1   =   2^52 + 2^51 + 2^50 + ... + 2^2 + 2^1 + 2^0
#
# After that, we can only get larger numbers by increasing the exponent, i.e. multiplying 
# previous numbers by powers of two. That means no further odd numbers can be represented
# exactly as Float64s! We can represent 2^53 exactly but not 2^53 + 1. So the range of
# integers that can be represented exactly as Float64's is -2^53 to 2^53. Let's verify.

2.0^53

9.007199254740992e15

In [14]:
# show that 2.0^53 is exactly the integer 2^53
Int64(2.0^53)  - 2^53  

0

In [16]:
# show that 2.0^53 - 1.0 is exactly the integer 2^53 - 1
Int64(2.0^53 - 1.0)  - (2^53 - 1)

0

In [17]:
# show that 2.0^53 + 1.0 is not the integer 2^53 + 1
Int64(2.0^53 + 1.0)  - (2^53 + 1)

-1

In [18]:
# Show all the same for -2^53
@show Int64(-2.0^53)  - (-2^53)
@show Int64(-2.0^53 + 1.0)  - (-2^53 + 1)
@show Int64(-2.0^53 - 1.0)  - (-2^53 - 1)

Int64(-(2.0 ^ 53)) - -(2 ^ 53) = 0
Int64(-(2.0 ^ 53) + 1.0) - (-(2 ^ 53) + 1) = 0
Int64(-(2.0 ^ 53) - 1.0) - (-(2 ^ 53) - 1) = 1


1

In [19]:
# This confirms that the range of exact integer Float64s is -2^53 to 2^53

## Problem 3

There is a debate underway in the Julia development community whether the time library should represent time with floating-point numbers or with integers. (https://discourse.julialang.org/t/why-do-time-quantities-have-to-be-integers/5864/51) Suppose you want to measure time with millisecond accuracy near the present, but also dates as far back as 0 BC, using just a single number. What kind of number should you use, integer or floating-point? How many bits would you need? Explain your reasoning. 

In [20]:
# Ok, how many milliseconds ago was 0 B.C.? 
# (1000 ms per s) * (60 s/min) * (60 min/hr) * (24 hr/day) * (365.24 day/yr) * (2017 yr)
1000*60*60*24*365.24*2017

6.3649936512e13

In [22]:
# Ok, 0 BC was 6 × 10¹³ milliseconds ago. So if we count in milliseconds with integers, 
# we need to be able to represent integers up to at least 6 × 10¹³ to go back to 0 BC

# Int32 spans 2^-31 to 2^31. Is 2^31 big enough?
2.0^31

2.147483648e9

In [23]:
# 2^31 ≈ 2e09 << 6e13, so Int32 is not big enough to count in milliseconds back to 0 BC

# Let's try Int64, which spans -2^63 to 2^63
2.0^63

9.223372036854776e18

In [24]:
# 2^63 ≈ 1e19 >> 6e13, so Int64 is big enough to count in milliseconds back to 0 BC
#
# Still, I would prefer to use a floating point number system to represent time, 
# since this would allow very high precision time stamps near the present (1e-308 seconds, for example)
# and very long times going into the future and past (±1e300 seconds, for example). 

## Problems 4-10

For each of problems 4 through 10, use the Conditioning and Accuracy Theorem to estimate a value for the expected relative error $|\tilde{f} - f|\,/\,|f|$ for the floating-point calculation $\tilde{f}(x)$ of the mathematical problem $f(x)$ in 64-bit arithmetic. Then devise a calculation in Julia that confirms your expectations. Explain each of your answers.

## Problem 4

**Multiplication, order-1 numbers.** Estimate a value for the relative error for floating-point calculation of multiplication $f(x) = cx$, with  $c=7/11$ and $x=19/3$ (numbers both close to 1). Confirm your estimate by calculating the relative error in Julia. Explain each answer.


## Problem 5

**Multiplication, large numbers.** Estimate a value for the relative error for floating-point calculation of multiplication, $f(x) = cx$, using $c=2/3 \times 10^{72}$ and $x=5/7 \times 10^{200}$ (two very large numbers). Confirm your estimate by calculating the relative error in Julia. Explain each answer.

## Problem 6

**Addition, order-1 numbers.** Estimate a value for the relative error for floating-point calculation of addition, $f(x_1, x_2) = x_1 + x_2$, using $x_1 = 17/3$ and $x_2 = 5/11$ (two numbers near 1). Confirm your estimate by calculating the relative error in Julia. Explain each answer.

## Problem 7

**Subtraction, very close numbers.** Estimate a value for the relative error for floating-point calculation of subtraction, $f(x_1, x_2) = x_2 - x_1$, using $x_1 = 129/11$ and $x_2 = x_1 + \delta$, where $\delta = 10^-13$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.



## Problem 8

**Powers, big $x$, small $n$.** Estimate a value for the relative error for floating-point calculation of the power function $f(x) = x^n$ for $x = 11/7 \times 10^{13}$ and $n=12$. Confirm your estimate by calculating the relative error in Julia. Explain each answer.

## Problem 9

**Powers, small $n$, big $x$.** Estimate a value for the relative error for floating-point calculation of the power function $f(x) = x^n$ for $x = 12$ and $n = 11/7 \times 10^{13}$. Compare your estimate to the relative error calculated in Julia. Explain each answer.

## Problem 10

**Solution of linear system.** Estimate a value for the relative error for numerical solution of the linear system $Ax=b$ for the given $A$ and $b$. Compare your estimate to a direct calculation of the relative error in Julia. Explain.


Note that in this case the data is $A,b$, the solution is $x$, the mathematical function is is $f(A,b) = A^{-1}b = x$, and the relative error is $|\tilde{x} - x\|\, / \, \|x\|$. The condition number of the $Ax=b$ solve is $\kappa(A) = \|A\| \|A^{-1}\|$, which can be calculated in Julia with `cond(A)`. 

In [29]:
function randA(m, κ)
    σ = logspace(0, -log10(κ), m)
    Σ = diagm(σ)
    U,tmp = qr(randn(m,m))
    V,tmp = qr(randn(m,m))
    A = U*Σ*V'
end
A = randA(5, 1e15);      # generate a random matrix A with condition number 1e15
x = randn(5);            # generate a random vector x, the true solution of Ax=b
b = A*x;                 # determine the right-hand-side vector b s.t. Ax=b

randA (generic function with 1 method)